In [ ]:
%cd /content/drive/MyDrive/LSBERT2
!pip install -r requirements.txt

/content/drive/MyDrive/LSBERT2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Succe

In [8]:
%cd /content/drive/MyDrive/LSBERT2

/content/drive/MyDrive/LSBERT2


In [77]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [78]:
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/LSBERT2')

In [79]:
importlib.reload(lsbert2)
importlib.reload(ranker)
importlib.reload(cwi_model)

<module 'cwi_model' from '/content/drive/MyDrive/LSBERT2/cwi_model.py'>

In [80]:
import importlib
import lsbert2
import ranker
import cwi_model

In [81]:
from lsbert2 import LSBertSimplifier
from load import load_subtlex, load_wiki_child, merge_frequencies, load_fasttext
from cwi_model import frequency_based_cwi, bert_cwi_func, ComplexWordIdentifier
import json
import joblib

In [8]:
fasttext_words, fasttext_vecs = load_fasttext("/content/drive/MyDrive/LSBERT2/embeddings/crawl-300d-2M.vec")

with open("/content/drive/MyDrive/LSBERT2/frequencies/freq_dict.json", "r") as f:
    freq_dict = json.load(f)

In [96]:
# Load cwi classifier
clf = joblib.load("/content/drive/MyDrive/LSBERT2/models/cwi_logistic_regression.joblib")

# Initialize complex word identifier
cwi = ComplexWordIdentifier(clf, freq_dict)

# Initialize simplifier
simplifier = LSBertSimplifier()
simplifier.word_freq_dict = freq_dict
simplifier.fasttext_dico = fasttext_words
simplifier.fasttext_emb = fasttext_vecs

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [157]:
# Parameters

num_candidates_per_word = 40
num_sentences = 10
alpha = 1.0 # lm score multiplier
beta = 1.0 # frequency score multiplier
gamma = 10.0 # similarity score multiplier

In [160]:
sentence = "The scientist, caught in a stupor of curiosity, postulated a groundbreaking theory based on anomalous experimental data."

#simplify_full_sentence(self, sentence, alpha = 1.0, beta = 1.0, gamma = 10.0, num_sentences=10, num_word_cands = 10, cwi_func=None):
simplified = simplifier.simplify_full_sentence(sentence, alpha = alpha, beta = beta, gamma = gamma, num_sentences = num_sentences, num_word_cands = num_candidates_per_word, cwi_func=lambda words: bert_cwi_func(words, cwi, threshold = 0.85))
final = simplifier.choose_most_similar(sentence, simplified)

print("Original sentence:")
print(sentence)
print("-----------------------")
print("\nSimplified sentence:")
print(final[0])

[{'word': 'stupor', 'index': 6}, {'word': 'curiosity', 'index': 8}, {'word': 'postulated', 'index': 10}, {'word': 'groundbreaking', 'index': 12}, {'word': 'anomalous', 'index': 16}, {'word': 'experimental', 'index': 17}]
Original sentence:
The scientist, caught in a stupor of curiosity, postulated a groundbreaking theory based on anomalous experimental data.
-----------------------

Simplified sentence:
The scientist , caught in a moment of hope , created a remarkable theory based on some test data .


In [159]:
sentence = "Researchers postulate that quantum fluctuations may have initiated the primordial inflation of the early universe."

#simplify_full_sentence(self, sentence, alpha = 1.0, beta = 1.0, gamma = 10.0, num_sentences=10, num_word_cands = 10, cwi_func=None):
simplified = simplifier.simplify_full_sentence(sentence, alpha = alpha, beta = beta, gamma = gamma, num_sentences = num_sentences, num_word_cands = num_candidates_per_word, cwi_func=lambda words: bert_cwi_func(words, cwi, threshold = 0.85))
final = simplifier.choose_most_similar(sentence, simplified)

print("Original sentence:")
print(sentence)
print("-----------------------")
print("\nSimplified sentence:")
print(final[0])

[{'word': 'postulate', 'index': 1}, {'word': 'quantum', 'index': 3}, {'word': 'fluctuations', 'index': 4}, {'word': 'primordial', 'index': 9}, {'word': 'inflation', 'index': 10}, {'word': 'universe', 'index': 14}]
Original sentence:
Researchers postulate that quantum fluctuations may have initiated the primordial inflation of the early universe.
-----------------------

Simplified sentence:
Researchers hold that cosmic information may have initiated the present events of the early years .


In [158]:
sentence = "Cognitive dissonance occurs when individuals experience psychological discomfort from holding contradictory beliefs."

#simplify_full_sentence(self, sentence, alpha = 1.0, beta = 1.0, gamma = 10.0, num_sentences=10, num_word_cands = 10, cwi_func=None):
simplified = simplifier.simplify_full_sentence(sentence, alpha = alpha, beta = beta, gamma = gamma, num_sentences = num_sentences, num_word_cands = num_candidates_per_word, cwi_func=lambda words: bert_cwi_func(words, cwi, threshold = 0.8))
final = simplifier.choose_most_similar(sentence, simplified)

print("Original sentence:")
print(sentence)
print("-----------------------")
print("\nSimplified sentence:")
print(final[0])

[{'word': 'Cognitive', 'index': 0}, {'word': 'dissonance', 'index': 1}, {'word': 'psychological', 'index': 6}, {'word': 'discomfort', 'index': 7}, {'word': 'contradictory', 'index': 10}]
Original sentence:
Cognitive dissonance occurs when individuals experience psychological discomfort from holding contradictory beliefs.
-----------------------

Simplified sentence:
thought confusion occurs when individuals experience mental anxiety from holding conflicting beliefs .
